In [1]:
import json
import subprocess
from datetime import datetime

## Get log stream list

In [4]:
def call_shell_script(script_path, *args, output_file=None):
    # Prepare the command with arguments
    command = [script_path] + list(args)

    # Call the shell script
    result = subprocess.run(command, capture_output=True, text=True)

    # Check if the script executed successfully
    if result.returncode == 0:
        print("Script executed successfully.")
        if output_file:
            # Write output to the specified file
            with open(output_file, 'w') as file:
                file.write(result.stdout)
            print(f"Output written to {output_file}")
        else:
            # Print output to console
            print("Output:\n", result.stdout)
    else:
        print("Script execution failed.")
        print("Error:\n", result.stderr)
   

script_path = './get-log-streams.sh'
aws_profile = 'acl-production'
aws_region = 'us-east-1'
log_group_name = 'projects-main'
log_stream_prefix = 'application/application'
output_file = 'application-log-streams.json'

call_shell_script(script_path, aws_profile, aws_region, log_group_name, log_stream_prefix, output_file=output_file)

Script executed successfully.
Output written to application-log-groups.json


In [ ]:


# Path to your JSON file containing the log stream descriptors
json_file_path = 'log_streams.json'

# Your specified time interval (example format: '2023-01-01 00:00:00')
start_time_str = 'your-start-time'
end_time_str = 'your-end-time'

# Convert time strings to timestamps
start_timestamp = int(datetime.strptime(start_time_str, '%Y-%m-%d %H:%M:%S').timestamp()) * 1000
end_timestamp = int(datetime.strptime(end_time_str, '%Y-%m-%d %H:%M:%S').timestamp()) * 1000

# Load JSON data
with open(json_file_path, 'r') as file:
    data = json.load(file)

# Filter log streams based on time interval
filtered_log_streams = [stream for stream in data['logStreams']
                        if stream['firstEventTimestamp'] <= end_timestamp and
                        stream['lastEventTimestamp'] >= start_timestamp]

# AWS CLI command to get log events
log_group_name = 'your-log-group-name'  # Replace with your log group name
for stream in filtered_log_streams:
    log_stream_name = stream['logStreamName']
    output_file_name = f"{log_stream_name.replace('/', '_')}.txt"
    command = f"aws logs get-log-events --log-group-name {log_group_name} --log-stream-name \"{log_stream_name}\" --start-time {start_timestamp} --end-time {end_timestamp} > {output_file_name}"
    subprocess.run(command, shell=True)

print("Logs downloaded for filtered streams.")
